In [1]:
import os
from zeroae.rocksdb import c as crocksdb

In [2]:
DBPath="/tmp/rocksdb_simple_example"
DBBackupPath="/tmp/rocksdb_simple_example_backup"

### Optimize RocksDB
The easiest way to get RocksDB to perform well -- use all your cores.

In [3]:
options = crocksdb.options.create()
crocksdb.options.increase_parallelism(options,os.cpu_count())
crocksdb.options.optimize_level_style_compaction(options,0)
crocksdb.options.set_create_if_missing(options, 1)

### Open the database

In [4]:
db = crocksdb.open(options, DBPath)

### Open the backup engine
We will use it for backing up our database.

In [5]:
be = crocksdb.backup_engine.open(options, DBBackupPath);

### Put key-value

In [6]:
writeoptions = crocksdb.writeoptions.create()
key = "key"
value = "value"
crocksdb.put(db, writeoptions, key, value)
crocksdb.writeoptions.destroy(writeoptions)

### Read the value back

In [7]:
readoptions = crocksdb.readoptions.create()
rv = crocksdb.get(db, readoptions, "key")
assert rv == "value"
crocksdb.readoptions.destroy(readoptions)

### Create a backup in DBBackupPath

In [8]:
crocksdb.backup_engine.create_new_backup(be, db)
crocksdb.close(db)

### Restore backup back into DBPath

In [9]:
restore_options = crocksdb.restore_options.create()
crocksdb.backup_engine.restore_db_from_latest_backup(
    be, DBPath, DBPath, restore_options
)
crocksdb.restore_options.destroy(restore_options)
crocksdb.backup_engine.close(be)

### Reopen the restored database

In [10]:
db = crocksdb.open(options, DBPath)

### Cleanup

In [ ]:
crocksdb.options.destroy(options)
crocksdb.close(db)